# Análise de dados da fila da creche no município de São Paulo

Obtendo os dados das matrículas na rede municipal de educação, obtidos [aqui](http://dados.prefeitura.sp.gov.br/dataset/demanda-por-vagas-educacao-infantil-e-eja):

In [1]:
import pandas as pd

dados_creche = pd.read_csv('dados/evolucaodemanda.csv', sep = ';')

Temos que filtrar apenas as colunas que necessárias. Além disso, é preciso limpar os dados, pois algumas linhas possuem os números com um ponto (.) para indicar o milhar. Por conveniência, o ano de referência será deixado em uma coluna a parte. 

In [2]:
for i in dados_creche.index:
    if not dados_creche.at[i, 'MAT_CRECHE'].is_integer():
        dados_creche.at[i, 'MAT_CRECHE'] *= 1000
    if not dados_creche.at[i, 'DEM_CRECHE'].is_integer():
        dados_creche.at[i, 'DEM_CRECHE'] *= 1000
    dados_creche.at[i, 'MES'] = dados_creche.at[i, 'MES_ANO_REF'][:3]
    dados_creche.at[i, 'ANO'] = int(dados_creche.at[i, 'MES_ANO_REF'][4:]) + 2000
    
dados_creche = dados_creche[['DISTRITO', 'MAT_CRECHE', 'DEM_CRECHE', 'MES', 'ANO', 'MES_ANO_REF']]

Agora, vamos obter os dados populacionais por distrito, com os dados da [SEADE](http://produtos.seade.gov.br/produtos/projpop/):

In [3]:
dados_pop = pd.read_csv('dados/populacao.csv', sep = ';', encoding = 'ISO-8859-1')

Para que os dados populacionais fiquem mais próximos aos da creche, vamos alterar o nome do distrito, deixando-o sem acentos e com caixa alta. Como os dados da população estão em números inteiros, não precisaremos remover os eventuais pontos (.) que indicam o milhar. Além disso, as colunas que selecionaremos terão os mesmos nomes nas nossas duas bases de dados (creche e população).

In [4]:
# Filtra as colunas
dados_pop = dados_pop[['Localidades', 'Períodos', 'População de 0 a 3 Anos']]
dados_pop.columns = ['DISTRITO_F', 'ANO', 'TOTAL']
    
# Tira os acentos dos nomes dos distritos e deixa o nome de cada distrito em caixa alta
dados_pop['DISTRITO'] = dados_pop['DISTRITO_F'].str.normalize('NFKD')
dados_pop['DISTRITO'] = dados_pop['DISTRITO'].str.encode('ascii', errors='ignore').str.decode('utf-8')
dados_pop['DISTRITO'] = dados_pop['DISTRITO'].str.upper()

Precisamos juntar os dados, para que a análise seja feita de maneira mais eficiente. Teremos um dicionário com os seguintes valores:

- Nome do distrito (em caixa alta e sem acentos);
- Número de matriculas em creches;
- Demanda de vagas em creches;
- Mês de referência (mmm);
- Ano de referência (aaaa);
- População do distrito na faixa etária 00 a 03 anos.

In [5]:
dados_creche_pop = dados_creche.merge(dados_pop, on = ['DISTRITO', 'ANO'])

## Visualização dos dados

### Gráficos

Para que a análise fique mais visual, vamos exibir alguns gráficos. Eles mostrarão, para cada distrito, a evolução do atendimento das creches municipais. Eles ficarão salvos na pasta `graficos`.

In [6]:
import matplotlib.pyplot as plt

for distrito in dados_creche_pop.DISTRITO.unique():
    dados = dados_creche_pop[dados_creche_pop['DISTRITO'] == distrito]
    plt.ioff()
    fig = plt.figure(dpi=400, figsize=(14, 6))
    plt.title('Evolução do atendimento das creches em ' + dados.at[dados.index[0], 'DISTRITO_F'])
    plt.plot(dados['MES_ANO_REF'], dados['MAT_CRECHE'], label='Matrículas nas creches do distrito', color='g')
    plt.plot(dados['MES_ANO_REF'], dados['DEM_CRECHE'], label='Demanda no distrito', color='r')
    plt.plot(dados['MES_ANO_REF'], dados['TOTAL'], label='Total da população no distrito', color='b')
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2),  shadow=True, ncol=2)
    plt.ylabel('Crianças de 0 a 3 anos')
    plt.xlabel('Data de Referência')
    plt.xticks(rotation=90)
    plt.savefig('graficos/' + distrito + '.png', bbox_inches='tight')
    plt.close(fig)

### Listas


Vamos também gerar algumas listas como parte de nossa análise de dados. Afinal, temos 96 gráficos, e precisamos extrair as principais informações dos dados. As listas que faremos serão:

- Atendimento (matrícula) absoluto por tempo.
- Atendimento (matrícula) relativo (relação à população) por tempo.
- Fila (demanda) absoluto por tempo.
- Fila (demanda) relativo por tempo.
- Variação entre junho de 2016 e dezembro de 2017 do atendimento absoluto.
- Variação entre junho de 2016 e dezembro de 2017 do atendimento relativo.

#### Lista do atendimento absoluto por tempo

Para essa lista, será gerado um arquivo .csv para cada período de referência (trimestre) com os distritos ordenados pelo número de matrículas.

In [6]:
# Gerando os arquivos
for periodo in dados_creche_pop.MES_ANO_REF.unique():
    dados = dados_creche_pop[dados_creche_pop['MES_ANO_REF'] == periodo]
    atendimento_absoluto = dados.sort_values(by='MAT_CRECHE', ascending=False)
    atendimento_absoluto = atendimento_absoluto[['DISTRITO', 'MAT_CRECHE']] 
    atendimento_absoluto.to_csv('resultados/atendimento_absoluto/atendimento_absoluto_' + periodo + '.csv', encoding='utf-8', index=False)

Também será gerado um gráfico mostrando a evolução do número de matrículas, comparando todos os distritos, mas talvez, fazendo um gráfico por zona, para ficar mais fácil de ver.

In [8]:
# Fazer gráficos por região/zona.

#### Lista do atendimento relativo por tempo

Para essa lista, será gerado um arquivo .csv para cada período de referência (trimestre) com os distritos ordenados pelo quociente matrículas por total populacional.

In [7]:
dados_creche_pop['ATEND_REL'] = dados_creche_pop['MAT_CRECHE'] / dados_creche_pop['TOTAL']

for periodo in dados_creche_pop.MES_ANO_REF.unique():
    dados = dados_creche_pop[dados_creche_pop['MES_ANO_REF'] == periodo]
    atendimento_relativo = dados.sort_values(by='ATEND_REL', ascending=False)
    atendimento_relativo = atendimento_relativo[['DISTRITO', 'ATEND_REL']] 
    atendimento_relativo.to_csv('resultados/atendimento_relativo/atendimento_relativo_' + periodo + '.csv', encoding='utf-8', index=False)

Também será gerado um gráfico mostrando a evolução do atendimento, comparando todos os distritos, mas talvez, fazendo um gráfico por zona, para ficar mais fácil de ver.

In [10]:
# Fazer gráficos por região/zona.

#### Lista da fila absoluta por tempo

Para essa lista, será gerado um arquivo .csv para cada período de referência (trimestre) com os distritos ordenados pelo número de crianças na fila.

In [8]:
# Gerando os arquivos
for periodo in dados_creche_pop.MES_ANO_REF.unique():
    dados = dados_creche_pop[dados_creche_pop['MES_ANO_REF'] == periodo]
    fila_absoluta = dados.sort_values(by='DEM_CRECHE', ascending=False)
    fila_absoluta = fila_absoluta[['DISTRITO', 'DEM_CRECHE']] 
    fila_absoluta.to_csv('resultados/fila_absoluta/fila_absoluta_' + periodo + '.csv', encoding='utf-8', index=False)

Também será gerado um gráfico mostrando a evolução do atendimento, comparando todos os distritos, mas talvez, fazendo um gráfico por zona, para ficar mais fácil de ver.

In [12]:
# Fazer gráficos por região/zona.

#### Lista do fila relativa por tempo

Para essa lista, será gerado um arquivo .csv para cada período de referência (trimestre) com os distritos ordenados pelo quociente demanda por total populacional.

In [9]:
dados_creche_pop['FILA_REL'] = dados_creche_pop['DEM_CRECHE'] / dados_creche_pop['TOTAL']

for periodo in dados_creche_pop.MES_ANO_REF.unique():
    dados = dados_creche_pop[dados_creche_pop['MES_ANO_REF'] == periodo]
    fila_relativa = dados.sort_values(by='FILA_REL', ascending=False)
    fila_relativa = fila_relativa[['DISTRITO', 'FILA_REL']] 
    fila_relativa.to_csv('resultados/fila_relativa/fila_relativa_' + periodo + '.csv', encoding='utf-8', index=False)

#### Lista da evolução absoluta do número de matrículas

Essa lista trará a variação, do número de matrículas, de junho de 2006 a dezembro de 2017. A última coluna trará a variação percentual desses valores.

In [28]:
dados_matricula = dados_creche_pop[['DISTRITO', 'MAT_CRECHE', 'MES_ANO_REF']]
dados_jun_2006 = dados_matricula[dados_matricula['MES_ANO_REF'] == 'jun-06']
dados_jun_2006.columns = ['DISTRITO', 'MAT_JUN_06', 'MES_ANO_REF']
dados_dez_2017 = dados_matricula[dados_matricula['MES_ANO_REF'] == 'dez-17']
dados_dez_2017.columns = ['DISTRITO', 'MAT_DEZ_17', 'MES_ANO_REF']

dados_matricula = dados_jun_2006.merge(dados_dez_2017, on = ['DISTRITO'])
dados_matricula = dados_matricula[['DISTRITO', 'MAT_JUN_06', 'MAT_DEZ_17']]

dados_matricula['EV_ABS_NUM'] = dados_matricula['MAT_DEZ_17'] - dados_matricula['MAT_JUN_06']
dados_matricula['EV_ABS_PC'] = dados_matricula['MAT_DEZ_17'] / dados_matricula['MAT_JUN_06']
# dados_matricula.sort_values(by='EV_ABS_NUM', ascending=False)
# dados_matricula.sort_values(by='EV_ABS_PC', ascending=False)

#### Lista da evolução absoluta da demanda

Essa lista trará a variação, em número de crianças, de junho de 2006 a dezembro de 2017. A última coluna trará a variação percentual desses valores.

In [38]:
dados_demanda = dados_creche_pop[['DISTRITO', 'DEM_CRECHE', 'MES_ANO_REF']]
dados_jun_2006 = dados_demanda[dados_demanda['MES_ANO_REF'] == 'jun-06']
dados_jun_2006.columns = ['DISTRITO', 'DEM_JUN_06', 'MES_ANO_REF']
dados_dez_2017 = dados_demanda[dados_demanda['MES_ANO_REF'] == 'dez-17']
dados_dez_2017.columns = ['DISTRITO', 'DEM_DEZ_17', 'MES_ANO_REF']

dados_demanda = dados_jun_2006.merge(dados_dez_2017, on = ['DISTRITO'])
dados_demanda = dados_demanda[['DISTRITO', 'DEM_JUN_06', 'DEM_DEZ_17']]

dados_demanda['EV_ABS_NUM'] = dados_demanda['DEM_DEZ_17'] - dados_demanda['DEM_JUN_06']
dados_demanda['EV_ABS_PC'] = dados_demanda['DEM_DEZ_17'] / dados_demanda['DEM_JUN_06']
# dados_demanda.sort_values(by='EV_ABS_NUM', ascending=True)
# dados_demanda.sort_values(by='EV_ABS_PC', ascending=True)